In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import sys
sys.path.append("../")
import src.Resources as src

In [2]:
movies = pd.read_csv('../Data/Movies_clean.csv', index_col= 0)  
books = pd.read_csv('../Data/Books_clean.csv', index_col= 0)  
books_2 = pd.read_csv('../Data/Books.csv',index_col=0)

In [3]:
books_2

,Book_author,Book_img,Book_description,Book_rating,Book_votes,Book_title,Book_genre
0,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Romeo and Juliet, Shakespeare creates a vio...",3.74,2462752,Romeo and Juliet,Classics|Plays|Fiction|Romance
1,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"Among Shakespeare's plays, ""Hamlet"" is conside...",4.02,888492,Hamlet,Classics|Plays|Fiction|Drama
2,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"One night on the heath, the brave and respecte...",3.90,836710,Macbeth,Classics|Plays|Fiction|School
3,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Othello, Shakespeare creates a powerful dra...",3.89,368863,Othello,Classics|Plays|Fiction|Drama
4,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,Shakespeare's intertwined love polygons begin ...,3.95,513517,A Midsummer Night's Dream,Classics|Plays|Fiction|Fantasy
...,...,...,...,...,...,...,...
1171,Stephenie Meyer,https://images-na.ssl-images-amazon.com/images...,"""BELLA?"" Edward's soft voice came from behind ...",3.72,1697723,Eclipse,YoungAdult|Fantasy|Romance|Vampires
1172,Samantha Kolesnik,https://images-na.ssl-images-amazon.com/images...,"Suzy and her brother, Lim, live with their abu...",3.67,2222,True Crime,Horror|Novella|Fiction|Thriller
1173,Susan Hill,https://images-na.ssl-images-amazon.com/images...,The Small Hand: A Ghost Story (The Susan Hill ...,3.46,6550,The Small Hand,Horror|Fiction|Gothic|GhostStories
1174,Carrie Ryan,https://images-na.ssl-images-amazon.com/images...,There are many things that Annah would like to...,4.02,22448,The Dark and Hollow Places,YoungAdult|Zombies|Dystopia|Horror


We drop any duplicates titles

In [4]:
movies["Stars"].fillna("Nostars",inplace=True)
movies.drop_duplicates(inplace=True)
movies = movies.drop_duplicates(subset="Title")
movies = movies.reset_index()
movies = movies.drop("index",axis=1)

books["Description"].fillna("No description",inplace=True)
books.drop_duplicates(inplace=True)
books = books.drop_duplicates(subset="Title")
books = books.reset_index()
books = books.drop("index",axis=1)

We split these columns by this character |

In [5]:
director = movies["Directors"].apply(lambda x: x.split("|"))
m_genre = movies["Genre"].apply(lambda x: x.split("|"))
stars = movies["Stars"].apply(lambda x: x.split("|"))
b_genre = books_2["Book_genre"].apply(lambda x: x.split("|"))

We explode some columns and get their unique values

In [6]:
n_directors = len(director.explode().unique())
movie_genres = len(m_genre.explode().unique())
movie_stars = len(stars.explode().unique())
book_genre = len(b_genre.explode().unique())
n_authors = books["Author"].nunique()
n_titles = books["Title"].nunique()
n_titles_m = movies["Title"].nunique()

These will be our big numbers

In [7]:
Bignumbers = pd.DataFrame([n_authors], columns=["Book_Authors"])
Bignumbers["Movie_directors"] = n_directors
Bignumbers["Movie_stars"] = movie_stars
Bignumbers["Book_titles"] = n_titles
Bignumbers["Movie_titles"] = n_titles_m
Bignumbers["Book_genres"] = book_genre
Bignumbers["Movie_genres"] = movie_genres

In [8]:
Bignumbers

,Book_Authors,Movie_directors,Movie_stars,Book_titles,Movie_titles,Book_genres,Movie_genres
0,5202,41338,129173,12473,77900,309,28


In [9]:
Bignumbers.to_csv("../Data/Bignumbers.csv")

We get some ratings for our graphs, i decided to filter by 1000 votes, feel free to filter higher or lower

In [10]:
movies_1 = movies[movies["Votes"]>1000]
books_1 = books[books["Votes"]>1000]

In [11]:
max_m =movies_1["Rating"].max()
min_m = movies_1["Rating"].min()
median_m = movies_1["Rating"].median()
mean_m = movies_1["Rating"].mean()

In [12]:
mean_m

6.3038062159839585

In [13]:
max_b = books_1["Rating"].max()
min_b = books_1["Rating"].min()
median_b = books_1["Rating"].median()
mean_b = books_1["Rating"].mean()

In [14]:
mean_b

8.010928386395921

These will be our metrics for our streamlit graphs

In [15]:
metric = ["Max","Min","Median","Mean","Max","Min","Median","Mean"]
rating = [max_m,min_m,median_m,mean_m,max_b,min_b,median_b,mean_b]
type = ["Movie","Movie","Movie","Movie","Book","Book","Book","Book"]


In [16]:
Metrics_movies_books = pd.DataFrame({"Metric": metric, "Rating": rating, "Type": type})
Metrics_movies_books.to_csv("../Data/Metrics_movies_books.csv")

In [17]:
books['Genre'] = books['Genre'].apply(src.map_genre)
movies["Genre"] = movies["Genre"].apply(src.map_movie_genre)

In [18]:
books.dropna(inplace=True)
movies.dropna(inplace=True)
books["Genre"] = books["Genre"].apply(lambda x: x.split("|"))
movies["Genre"] = movies["Genre"].apply(lambda x : x.split("|"))

In [19]:
genre_books = books["Genre"].explode().value_counts().reset_index()
genre_movies = movies["Genre"].explode().value_counts().reset_index()

In [20]:
genre_books["Type"]= "Book"
genre_movies["Type"] = "Movie"

In [21]:
genre_total = pd.concat([genre_books,genre_movies])

Here i want to filter with votes so i only get the best books and movies, this will be used to do a filter that shows the 3 best movies and books of each genre

In [22]:
movies_2 = movies[movies["Votes"]>80000]
books_2 = books[books["Votes"]>80000]

In [23]:
books_exploded = pd.DataFrame({"Image":books_2["Image"],"Title": books_2["Title"],"Author":books_2["Author"], "Rating": books_2["Rating"],"Description":books_2["Description"],"Genre":books_2["Genre"]})
movies_exploded = pd.DataFrame({"Image":movies_2["Image"],"Title": movies_2["Title"],"Directors":movies_2["Directors"], "Rating": movies_2["Rating"],"Description": movies_2["Description"],"Genre":movies_2["Genre"]})


In [24]:
books_exploded = books_exploded.explode("Genre").reset_index(drop=True)
movies_exploded = movies_exploded.explode("Genre").reset_index(drop=True)

In [25]:
books_exploded

,Image,Title,Author,Rating,Description,Genre
0,https://images-na.ssl-images-amazon.com/images...,Romeo and Juliet,William Shakespeare,7.48,"In Romeo and Juliet, Shakespeare creates a vio...",Romance
1,https://images-na.ssl-images-amazon.com/images...,Hamlet,William Shakespeare,8.04,"Among Shakespeare's plays, ""Hamlet"" is conside...",Drama
2,https://images-na.ssl-images-amazon.com/images...,Macbeth,William Shakespeare,7.80,"One night on the heath, the brave and respecte...",Family
3,https://images-na.ssl-images-amazon.com/images...,Othello,William Shakespeare,7.78,"In Othello, Shakespeare creates a powerful dra...",Drama
4,https://images-na.ssl-images-amazon.com/images...,A Midsummer Night's Dream,William Shakespeare,7.90,Shakespeare's intertwined love polygons begin ...,Fantasy
...,...,...,...,...,...,...
5242,https://images-na.ssl-images-amazon.com/images...,Curious George,H.A. Rey,8.32,The first adventure in this highly popular ser...,Family
5243,https://images-na.ssl-images-amazon.com/images...,The Hard Thing About Hard Things: Building a B...,Ben Horowitz,8.44,A lot of people talk about how great it is to ...,Biography
5244,https://images-na.ssl-images-amazon.com/images...,The Hard Thing About Hard Things: Building a B...,Ben Horowitz,8.44,A lot of people talk about how great it is to ...,History
5245,https://images-na.ssl-images-amazon.com/images...,The Princess Saves Herself in This One,Amanda Lovelace,7.68,"""Ah, life- the thing that happens to us while ...",Biography


In [26]:
books_exploded.to_csv("../Data/Books_exploded.csv")
movies_exploded.to_csv("../Data/Movies_exploded.csv")
genre_total.to_csv("../Data/genre_total.csv")